## 키워드 검색량 도구 API test

In [29]:
import random
import time
from datetime import datetime 
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import json
import signaturehelper
import requests

In [30]:
config_secret_debug = json.loads(open(".config_secret/settings_debug.json").read())
customer_id = config_secret_debug["NAVER_SEARCH"]["CUSTOMER_ID"]
client_id = config_secret_debug["NAVER_SEARCH"]["CLIENT_ID"]
client_secret = config_secret_debug["NAVER_SEARCH"]["CLIENT_SECRET"]

In [31]:
## 입력

keyword = input("키워드 입력: ")
start_date = "2016-01-01"
end_date = str(datetime.now()-timedelta(days=1))[:10]
time_unit = "date"
keyword_groups = keyword
print(start_date, end_date)

키워드 입력: 폼클렌징
2016-01-01 2021-02-09


In [32]:
def get_search_header(method, uri, api_key, secret_key, customer_id):
    timestamp = str(round(time.time() * 1000))
    signature = signaturehelper.Signature.generate(timestamp, method, uri, SECRET_KEY)
    
    return {'Content-Type': 'application/json; charset=UTF-8', 
            'X-Timestamp': timestamp, 
            'X-API-KEY': API_KEY, 
            'X-Customer': str(CUSTOMER_ID),
            'X-Signature': signature}

In [33]:
def get_datalab_header(api_key, secret_key):
    return {
        "Content-Type":"application/json",
        "X-Naver-Client-Id": api_key,
        "X-Naver-Client-Secret":secret_key        
    }

In [34]:
BASE_URL = 'https://api.naver.com'
API_KEY = client_id
SECRET_KEY = client_secret
CUSTOMER_ID = customer_id

In [35]:
uri = '/keywordstool'
method = 'GET'
search_r = requests.get(BASE_URL + uri+'?hintKeywords={}&showDetail=1'.format(keyword),
                 params = {"sort":"date"},
                 headers=get_search_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))

In [36]:
search_result = search_r.json()["keywordList"][0]
search_result

{'relKeyword': '폼클렌징',
 'monthlyPcQcCnt': 6140,
 'monthlyMobileQcCnt': 41800,
 'monthlyAvePcClkCnt': 13.2,
 'monthlyAveMobileClkCnt': 204.8,
 'monthlyAvePcCtr': 0.24,
 'monthlyAveMobileCtr': 0.53,
 'plAvgDepth': 15,
 'compIdx': '높음'}

In [37]:
import os
import sys
import urllib.request

In [38]:
client_id =  config_secret_debug["NAVER"]["CLIENT_ID"]
client_secret = config_secret_debug["NAVER"]["CLIENT_SECRET"]
url = "https://openapi.naver.com/v1/datalab/search"
body = { "startDate":start_date,
         "endDate":end_date,
         "timeUnit":time_unit,
         "keywordGroups" : [{"groupName": keyword, "keywords" : [keyword]}]
       }

In [39]:
datalab_r = requests.post(url,data = json.dumps(body),
                 headers=get_datalab_header(client_id, client_secret))

In [40]:
datalab_result = json.loads(datalab_r.content)
datalab_result

{'startDate': '2016-01-01',
 'endDate': '2021-02-09',
 'timeUnit': 'date',
 'results': [{'title': '폼클렌징',
   'keywords': ['폼클렌징'],
   'data': [{'period': '2016-01-01', 'ratio': 47.66626},
    {'period': '2016-01-02', 'ratio': 46.98239},
    {'period': '2016-01-03', 'ratio': 48.16207},
    {'period': '2016-01-04', 'ratio': 44.94785},
    {'period': '2016-01-05', 'ratio': 40.29748},
    {'period': '2016-01-06', 'ratio': 45.28979},
    {'period': '2016-01-07', 'ratio': 43.34074},
    {'period': '2016-01-08', 'ratio': 43.30654},
    {'period': '2016-01-09', 'ratio': 44.75978},
    {'period': '2016-01-10', 'ratio': 43.06719},
    {'period': '2016-01-11', 'ratio': 48.53821},
    {'period': '2016-01-12', 'ratio': 49.94016},
    {'period': '2016-01-13', 'ratio': 45.85399},
    {'period': '2016-01-14', 'ratio': 48.53821},
    {'period': '2016-01-15', 'ratio': 50.2308},
    {'period': '2016-01-16', 'ratio': 44.70849},
    {'period': '2016-01-17', 'ratio': 47.56368},
    {'period': '2016-01-18', 

### 최근 30일간 검색량 

In [14]:
import pandas as pd

In [164]:
month_ago = str(datetime.now()-relativedelta(months=1))[:10]
current_date = str(datetime.now())[:10]

In [15]:
dlab_list = datalab_result["results"][0]["data"]
month_index = next((index for (index, d) in enumerate(dlab_list) if d["period"] == month_ago), None)
dlab_list = dlab_list[month_index:]
ratio_sum = sum(dlab_list[i]["ratio"] for i in range(len(dlab_list)))
ratio_sum

760.3691399999999

In [28]:
print(str(datetime.now()-relativedelta(months=1))[:10])

2021-01-10


In [16]:
df = pd.DataFrame(search_result, index = ["values"])
df.drop(["compIdx",
         "plAvgDepth",
         "monthlyAveMobileCtr",
         "monthlyAvePcCtr",
         "monthlyAveMobileClkCnt",
         "monthlyAvePcClkCnt"], axis = 1 ,inplace = True)
df.rename(columns={"relKeyword":"검색어","monthlyPcQcCnt":"PC", "monthlyMobileQcCnt":"Mobile"}, inplace = True)
df["Total"] = search_result["monthlyPcQcCnt"] + search_result["monthlyMobileQcCnt"]
df["전월 대비"] = "_"

In [17]:
df["Total"] / ratio_sum

values    63.048324
Name: Total, dtype: float64

In [18]:
display(df)

,검색어,PC,Mobile,Total,전월 대비
values,폼클렌징,6140,41800,47940,_


### 최근 컨텐츠 발행량

In [52]:
encText = urllib.parse.quote(keyword)
url_blog = "https://openapi.naver.com/v1/search/blog"
url_cafe = "https://openapi.naver.com/v1/search/cafearticle?query=" + encText

In [81]:
content_blog_r = requests.get(url_blog,
                              params = {"sort":"sim", "query" : encText, "display":100},
                              headers=get_datalab_header(client_id, client_secret))
content_cafe_r = requests.get(url_cafe,
                              headers=get_datalab_header(client_id, client_secret))

In [82]:
blog_result = json.loads(content_blog_r.content)
cafe_result = json.loads(content_cafe_r.content)

In [165]:
cafe_url = "https://cafe.naver.com/ca-fe/home/search/articles"+"?q="+encText+"&pr=3"
blog_url = "https://section.blog.naver.com/Search/Post.nhn"+"?pageNo=1"+"&rangeType=MONTH"+"&orderBy=sim"+"&startDate="+month_ago+"&endDate="+current_date+"&keyword="+encText

In [101]:
### Test url

import webbrowser
webbrowser.open(blog_url)
webbrowser.open(cafe_url)

True

Selenium + Django 어떻게 연결할 껀지 고민해보기

In [151]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import pathlib as p

In [166]:
driver = webdriver.Chrome("./chromedriver_win32/chromedriver.exe")
driver.get(blog_url)
driver.implicitly_wait(3)
html = driver.page_source

In [167]:
soup = BeautifulSoup(html, "html.parser")
get_blog_number = soup.find("em",{"class","search_number"})
get_blog_number = int(get_blog_number.text[:-1].replace(",",""))
get_blog_number

15458

In [154]:
driver.get(cafe_url)
driver.implicitly_wait(3)

In [157]:
action = ActionChains(driver)
driver.find_element_by_id("app").click()
get_cafe_number = driver.find_element_by_class_name("total_count")
time.sleep(1)
get_cafe_number = int(get_cafe_number.text[:-1].replace(",",""))
print(get_cafe_number)

11132


In [ ]:
driver.close()

In [169]:
df = pd.DataFrame(index = ["values"])
df["Blog"] = get_blog_number
df["Cafe"] = get_cafe_number
df["Total"] = get_blog_number + get_cafe_number
df

,Blog,Cafe,Total
values,15458,11132,26590


### Graph

In [24]:
config_secret_debug = json.loads(open(".config_secret/settings_debug.json").read())
customer_id = config_secret_debug["NAVER"]["CUSTOMER_ID"]
client_id = config_secret_debug["NAVER"]["CLIENT_ID"]

KeyError: 'CUSTOMER_ID'

In [25]:
url = "https://openapi.naver.com/v1/datalab/search"

In [26]:
uri = '/keywordstool'
method = 'GET'
keyword = input("키워드 입력: ")
current_r = requests.get(BASE_URL + uri+'?hintKeywords={}&showDetail=1'.format(keyword),
                 headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))

키워드 입력: 폼클렌징


NameError: name 'get_header' is not defined

### 관련 키워드 현황

In [27]:
search_result

{'relKeyword': '폼클렌징',
 'monthlyPcQcCnt': 6140,
 'monthlyMobileQcCnt': 41800,
 'monthlyAvePcClkCnt': 13.2,
 'monthlyAveMobileClkCnt': 204.8,
 'monthlyAvePcCtr': 0.24,
 'monthlyAveMobileCtr': 0.53,
 'plAvgDepth': 15,
 'compIdx': '높음'}

In [ ]:
df = pd.DataFrame(search_result, index = ["values"])
df.drop(["compIdx",
         "plAvgDepth"])
